In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd 
import numpy as np 
import sklearn 

In [ ]:
train = pd.read_csv('../input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('../input/costa-rican-household-poverty-prediction/test.csv')

In [ ]:
map = {'yes': 1, 'no': 0}

for df in [train, test]:
    # Fill in the values with the correct mapping
    df['dependency'] = df['dependency'].replace(map).astype(float)
    df.loc[df['edjefa'] == "yes", "edjefa"] = 1
    df.loc[df['edjefe'] == "yes", "edjefe"] = 1
    df.loc[df['edjefa'] == "no", "edjefa"] = 0
    df.loc[df['edjefe'] == "no", "edjefe"] = 0
    df['edjefa'] = df['edjefa'].astype(int)
    df['edjefe'] = df['edjefe'].astype(int)
    df['v2a1']=df['v2a1'].fillna(0)
    df['v18q1']=df['v18q1'].fillna(0)
    df['rez_esc']=df['rez_esc'].fillna(0)
    df['meaneduc']=df['meaneduc'].fillna(0)
    df['SQBmeaned']=df['SQBmeaned'].fillna(0)

In [ ]:
train["Target"].value_counts()
class1_percentage = 755/ (5996 + 1597 + 1209 + 755)
class2_percentage = 1597/ (5996 + 1597 + 1209 + 755)
class3_percentage = 1209/ (5996 + 1597 + 1209 + 755)
class4_percentage = 5996/ (5996 + 1597 + 1209 + 755)

class1_weight = class1_percentage/class1_percentage
class2_weight = class1_percentage/class2_percentage
class3_weight = class1_percentage/class3_percentage
class4_weight = class1_percentage/class4_percentage

weight_dictionary = {4: class4_weight,
                     3: class3_weight,
                     2: class2_weight,
                     1: class1_weight}
print(class1_weight)
print(class4_weight)

In [ ]:

class_weights = train['Target'].map(weight_dictionary)


In [ ]:
# features = [c for c in train.columns if c not in ['Id', 'idhogar', 'Target']]
# from sklearn.ensemble import RandomForestClassifier

# rf= RandomForestClassifier(max_depth=27, random_state=42,
#                            n_estimators=707, min_impurity_decrease=0.0007, min_samples_leaf=1,
#                            min_samples_split=5, verbose=0, class_weight='balanced')

# rf.fit(train[features],train['Target'])

In [ ]:
features = [c for c in train.columns if c not in ['Id', 'idhogar', 'Target']]
from xgboost import XGBClassifier


xgbc=XGBClassifier(base_score=0.5, booster='gbtree', weight= class_weights, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=5, missing=None,
       n_estimators=250, n_jobs=1, nthread=None,
       objective='multi:softmax', random_state=0, reg_alpha=1,
       reg_lambda=1, seed=None, silent=True,
       subsample=0.5, verbosity=0) 


xgbc.fit(train[features],train['Target'])


In [ ]:
#Test Predictions
ids=test['Id']
test.drop(['Id', 'idhogar'], axis=1, inplace=True)

test_predictions=xgbc.predict(test).astype(int)
test_predictions[:5]
np.unique(test_predictions,return_counts=True)

In [ ]:
#Train Predictions
train_ids=train['Id']
train_target = train['Target']
train.drop(['Id', 'idhogar','Target'], axis=1, inplace=True)

train_predictions=xgbc.predict(train).astype(int)
train_predictions[:5]
np.unique(train_predictions,return_counts=True)


In [ ]:
from sklearn.metrics import f1_score
f1_score(train_target,  train_predictions, average='micro')

In [ ]:
submit=pd.DataFrame({'Id': ids, 'Target': test_predictions})
submit.to_csv('submission.csv', index=False)